In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
from time import sleep # this is to create a time delay

# 1. Start by using either the inspector or by viewing the page source. Can you identify a tag that might be helpful for finding the names of all performers? For now, just worry about the headliner and don't worry about the opener. (Eg. For Vince Gill, featuring Wendy Moten, we only care about Vince Gill.) Make use of this to create a list containing just the names of each inductee.

In [ ]:
url = 'https://www.ryman.com/events/'
response = requests.get(url)
response

In [ ]:
soup = BS(response.text)

In [ ]:
divs = soup.findAll('div', {'class':'info'})

#divs

In [ ]:
performers = []

for div in divs:
    try:
        performer = div.find('h3').text.strip()
        if performer != '':
            performers.append(performer)
            print(performer)
    except:
        continue

In [ ]:
performers

In [ ]:
divs[8].findAll('h3')

In [ ]:
divs[8].find('h3').text.strip()

# 2. Next, try and find a tag that could be used to find the date and time for each show. Extract these into a list. Challenge: Convert these into two lists, one containing the date and the other containing the time. (Eg. split Mar 9, 2023 8:00 PM into Mar 9, 2023 and 8:00 PM.)

In [ ]:
date_divs = soup.findAll('div', {'class':'date'})
#date_divs

In [ ]:
len(date_divs)

In [ ]:
dates = []
times = []

for div in date_divs:
    try:
        month = div.find('span', {'class':'m-date__month'}).text.strip()
        days  = div.findAll('span', {'class':'m-date__day'})#.text.strip()
        year  = div.find('span', {'class':'m-date__year'}).text.strip()
        time = ''
        
        if len(days) != 1:
            day = ''
            for d in days:
                day = day + ' ' + d.text
            day = day.strip().replace(' ', '-')
            
            time = 'Multiple times'
            
        else:
            day = days[0].text.strip()
            time  = div.find('span', {'class':'m-date__hour'}).text.strip()
        #end
        date = month+' '+day+year
        
        times.append(time)
        dates.append(date)     
        
        print(date, time)
    except:
        continue

In [ ]:
dates

In [ ]:
times

# 3. Take the lists you created on parts 1 and 2 and convert them into a pandas DataFrame.

In [ ]:
ryman_df = pd.DataFrame(performers)
ryman_df.columns = ['Performers']
ryman_df

In [ ]:
ryman_df['Date'] = dates
ryman_df['Time'] = times
ryman_df

# 4. Bonus #1: Add to your data frame the opening act for all shows that list an opener.

In [ ]:
event_divs = soup.findAll('div', {'class':'eventItem'})
event_divs.remove(event_divs[0])
#event_divs

In [ ]:
event_divs[5].h4.text.strip()

In [ ]:
openers = []

for div in event_divs:
    try:
        opener = div.h4.text.strip()
        openers.append(opener)
    except:
        openers.append('')
openers

In [ ]:
ryman_df['Openers and Info'] = openers
ryman_df

# 5. Bonus #2:: Now, let's see if we can get the results beyond the first page. For this, you'll need to Web Developer Tools of your browser and navigate to the Network tab. Click the "Load More Events" button and you should see a GET request to the www.ryman.com domain.
## a. Inspect this request and you should see that it goes to a URL like "https://www.ryman.com/events/events_ajax/24?category=0&venue=0&team=0&exclude=&per_page=12&came_from_page=event-list-page". In your Jupyter notebook, send a get request to this url and inspect the results.

In [ ]:
url = 'https://www.ryman.com/events/events_ajax/24?category=0&venue=0&team=0&exclude=&per_page=12&came_from_page=event-list-page'
response = requests.get(url)
response

In [ ]:
soup = BS(response.text)
#soup

## b. You should find that the results that you get are HTML, but that they are not exactly formatted in a way that can be parsed. See if you can clean up the results set so that you can extract out the same information as above

In [ ]:
text = response.text.replace('\\n','').replace('\\t','').replace('\\','').replace('\<\>','\< \>')
#text

In [ ]:
soup = BS(text)
print(soup.prettify)

In [ ]:
info = soup.findAll('div')
info

In [ ]:
len(info)

In [ ]:
performers = []
openers = []
dates = []
times = []
i = 0

for line in info:
    if i%7 == 2:
        performer = line.text.strip()
        try:
            text = performer.split('Auditorium')[1]
            text = text[:-20]
            #print('Text (1st try) is:',text)
            performer = text.split('       ')[0].strip()
            print(performer)
            
            try:
                opener = text.split('       ')[1].strip()
                print('Opener is:',opener)
                openers.append(opener)
            except:
                opener = ''
                print('No opener')
                openers.append(opener)
            #end
            
        except:
            try:
                text = performer.split('Opry House')[1]
                text = text[:-20]
                #print('Text (2nd try) is:',text)
                performer = text.split('       ')[0].strip()
                print(performer)
                
                try:
                    opener = text.split('       ')[1].strip()
                    print('Opener is:',opener)
                    openers.append(opener)
                except:
                    opener = ''
                    print('No opener')
                    openers.append(opener)
                #end
    
            except:
                print('EXCEPTION')
            #end
        #end
        performers.append(performer)
        
    
    if i%7 == 3:
        datetime = line.text
        pattern = re.compile('\d+:\d0\s.M')
        
        try:
            time = pattern.search(datetime).group(0).strip()
            date = datetime.split(time)[0].strip()
        except:
            time = 'Multiple times'
            date = datetime.strip()
        
        
        dates.append(date)
        times.append(time)
        print(date)
        print(time,'\n')
        
    
    i += 1


In [ ]:
openers

In [ ]:
dates

In [ ]:
times

In [ ]:
ryman_df2 = pd.DataFrame({'Performers':performers, 'Date':dates, 'Time':times, 'Opener and Info':openers})
ryman_df2

## c. Create a DataFrame that contains data for the next 60 shows.

In [ ]:
def get_ryman(url):
    response = requests.get(url)
    soup = BS(response.text)
            
            
    performers = []
    dates = []
    times = []
    openers = []
            

    divs = soup.findAll('div', {'class':'info'})
            
            
    ### Find performers
            
    for div in divs:
        try:
            performer = div.find('h3').text.strip()
            if performer != '':
                performers.append(performer)
        except:
            continue
    #end
            
    ### Find time componeents
            
    date_divs = soup.findAll('div', {'class':'date'})

    for div in date_divs:
        try:
            month = div.find('span', {'class':'m-date__month'}).text.strip()
            days  = div.findAll('span', {'class':'m-date__day'})#.text.strip()
            year  = div.find('span', {'class':'m-date__year'}).text.strip()
            time = ''

            if len(days) != 1:
                day = ''
                for d in days:
                    day = day + ' ' + d.text
                day = day.strip().replace(' ', '-')

                time = 'Multiple times'

            else:
                day = days[0].text.strip()
                time  = div.find('span', {'class':'m-date__hour'}).text.strip()
            #end
            date = month+' '+day+year

            times.append(time)
            dates.append(date)     
                    
        except:
            continue
    #end
            
    event_divs = soup.findAll('div', {'class':'eventItem'})
    event_divs.remove(event_divs[0])
            
    ### Find openers
            
    for div in event_divs:
        try:
            opener = div.h4.text.strip()
            openers.append(opener)
        except:
            openers.append('')
    #end
            
    
    ### We now will "load" more events
    
    for i in range(2,7):
        url = f'https://www.ryman.com/events/events_ajax/{i*12}?category=0&venue=0&team=0&exclude=&per_page=12&came_from_page=event-list-page'
        
        response = requests.get(url)
        text = response.text.replace('\\n','').replace('\\t','').replace('\\','').replace('\<\>','\< \>')
        soup = BS(text)
        
        info = soup.findAll('div')
    
        j = 0

        for line in info:
            
            ### Get performer and opener of the new events
            
            if j%7 == 2:
                performer = line.text.strip()
                try:
                    text = performer.split('Auditorium')[1]
                    text = text[:-20]
                    performer = text.split('       ')[0].strip()

                    try:
                        opener = text.split('       ')[1].strip()
                        openers.append(opener)
                    except:
                        opener = ''
                        openers.append(opener)
                    #end

                except:
                    try:
                        text = performer.split('Opry House')[1]
                        text = text[:-20]
                        performer = text.split('       ')[0].strip()

                        try:
                            opener = text.split('       ')[1].strip()
                            openers.append(opener)
                        except:
                            opener = ''
                            openers.append(opener)
                        #end

                    except:
                        print('EXCEPTION')
                    #end
                #end
                performers.append(performer)

            ### Get time components of new events
            
            if j%7 == 3:
                datetime = line.text
                pattern = re.compile('\d+:\d0\s.M')

                try:
                    time = pattern.search(datetime).group(0).strip()
                    date = datetime.split(time)[0].strip()
                except:
                    time = 'Multiple times'
                    date = datetime.strip()


                dates.append(date)
                times.append(time)


            j += 1
        #end
    
    ryman_df = pd.DataFrame({'Performer':performers, 'Date':dates, 'Time': times, 'Opener and Info':openers})
    
    return ryman_df

In [ ]:
get_ryman('https://www.ryman.com/events/')

## So close! When I ran it the first time, I got 60 rows, so there's something there!